In [1]:
import os
import xmlrpc.client
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [2]:
def api_params_func(test_db: bool = False) -> dict:

    api_url = os.environ.get('ODOO_URL_API')
    api_db = os.environ.get('ODOO_DB_API')
    api_test_db = os.environ.get('ODOO_DB_PRUEBA_API')
    api_username = os.environ.get('ODOO_USERNAME_API')
    api_clave = os.environ.get('ODOO_CLAVE_API')


    api_params = {}
    if test_db:
        api_params['api_db'] = api_test_db
    else:
        api_params['api_db'] = api_db


    common = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/common')
    uid = common.authenticate(api_params['api_db'], api_username, api_clave, {})
    models = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/object')


    api_params['api_clave'] = api_clave
    api_params['api_uid'] = uid
    api_params['api_models'] = models

    return api_params

In [3]:
def search_pay_func(mes: int) -> list[str]:
    
    if type(mes) != int or mes < 1 or mes > 12:
        raise Exception (f'El mes es incorrecto. El párametro "mes" debe ser un número entero entre 1 y 12. Escribiste: {mes}')
    
    param_dia_ini = datetime(2024, mes, 1)
    param_dia_fin = datetime(2024, mes + 1, 1) - timedelta(days= 1)

    return param_dia_ini.strftime('%Y-%m-%d'), param_dia_fin.strftime('%Y-%m-%d')

In [4]:
def api_call_pay_acc_func(api_params: dict, search_pay: list[str] ) -> list[dict]:
    
    api_db = api_params['api_db']
    api_clave = api_params['api_clave']
    uid = api_params['api_uid']
    models = api_params['api_models']


    param_dia_ini,  param_dia_fin = search_pay

    search_pay_acc = [
        "&", "&",
            ("partner_type", "=", "customer"),
            ("is_internal_transfer", "=", False),
        "&",
            ("date", ">=", param_dia_ini),
            ("date", "<=", param_dia_fin),
        ]


    pay_acc_fields = [
                    'name',
                    'date',
                    'amount_total',
                    'reconciled_invoice_ids',
                    'pos_session_id'
                    ]

    pay_acc_ids = models.execute_kw(api_db, uid, api_clave, 'account.payment', 'search', [search_pay_acc])
    pay_acc_json = models.execute_kw(api_db, uid, api_clave, 'account.payment', 'read', [pay_acc_ids], {'fields': pay_acc_fields})

    

    return pay_acc_json

In [7]:
def pay_acc_df_fun(pay_acc_json: list[dict]) -> list[pd.DataFrame, int]:
    
    data_pay_acc = []
    data_fact_ids = []

    for pay in pay_acc_json:
        if not pay['pos_session_id']:
            new = {}
            new['id'] = pay['id']
            new['name'] = pay['name']
            new['date'] = pay['date']
            new['amount_total'] = pay['amount_total']
            new['pay_fact_docs'] = pay['reconciled_invoice_ids'] if pay['reconciled_invoice_ids'] else pd.NA
            
            data_fact_ids += pay['reconciled_invoice_ids']

            data_pay_acc.append(new)

    pay_acc_df = pd.DataFrame(data_pay_acc)


    pay_acc_df['date'] = pd.to_datetime(pay_acc_df['date'], format='%Y-%m-%d')
    
    return pay_acc_df, list(set(data_fact_ids))

In [5]:
def api_call_pay_pos_func(api_params: dict, search_pay: list[str] ) -> list[dict]:
    
    api_db = api_params['api_db']
    api_clave = api_params['api_clave']
    uid = api_params['api_uid']
    models = api_params['api_models']


    param_dia_ini,  param_dia_fin = search_pay

    search_pay_pos = [
        "&",
            ("payment_date", ">=", param_dia_ini),
            ("payment_date", "<=", param_dia_fin),
        ]


    pay_pos_fields = [
                    'pos_order_id',
                    'payment_date',
                    'amount',
                    'payment_method_id'
                    ]

    pay_pos_ids = models.execute_kw(api_db, uid, api_clave, 'pos.payment', 'search', [search_pay_pos])
    pay_pos_json = models.execute_kw(api_db, uid, api_clave, 'pos.payment', 'read', [pay_pos_ids], {'fields': pay_pos_fields})

    

    return pay_pos_json

In [8]:
def pay_pos_df_fun(pay_pos_json: list[dict]) -> list[pd.DataFrame, int]:
    
    data_pay_pos = []
    data_pos_ids = set()

    for pay in pay_pos_json:
        if pay['payment_method_id'][0] != 6:
            new = {}
            new['pay_pos_id'] = pay['id']
            new['pos_order_id'] = pay['pos_order_id'][0]
            new['pos_order_name'] = pay['pos_order_id'][1]
            new['date'] = pay['payment_date']
            new['amount'] = pay['amount']
        
            data_pos_ids.add(pay['pos_order_id'][0])
            data_pay_pos.append(new)

    pay_pos_df1 = pd.DataFrame(data_pay_pos)
    pay_pos_df = pay_pos_df1.groupby('pos_order_id').agg({'pay_pos_id':['first'], 'pos_order_name':['first'], 'date':['first'], 'amount':['sum']}).reset_index().iloc[:,[1,0,2,3,4]]
    pay_pos_df.columns = ['pay_pos_id', 'pos_order_id', 'pos_order_name', 'date', 'amount']

    pay_pos_df['date'] = pd.to_datetime(pay_pos_df['date'], format='%Y-%m-%d %H:%M:%S')
    
    return pay_pos_df, list(data_pos_ids)

In [14]:
def api_call_pos_order_func(api_params: dict, data_pos_ids: list[int] ) -> list[dict]:
    
    api_db = api_params['api_db']
    api_clave = api_params['api_clave']
    uid = api_params['api_uid']
    models = api_params['api_models']

    pos_order_fields = [
                    'account_move',
                    'partner_id',
                    'session_move_id'
                    ]

    pos_order_json = models.execute_kw(api_db, uid, api_clave, 'pos.order', 'read', [data_pos_ids], {'fields': pos_order_fields})

    return pos_order_json

In [9]:
api_params = api_params_func(True)
search_pay = search_pay_func(4)

pay_acc_json = api_call_pay_acc_func(api_params, search_pay)
pay_pos_json = api_call_pay_pos_func(api_params, search_pay)

pay_acc_df, data_fact_ids = pay_acc_df_fun(pay_acc_json)
pay_pos_df, data_pos_ids = pay_pos_df_fun(pay_pos_json)

In [15]:
api_call_pos_order_func(api_params, data_pos_ids)

[{'id': 11241,
  'account_move': [52465, 'F1-CC/2024/04670 (PdV CSL/4627)'],
  'partner_id': [16031, 'JUAN JOSE ALFARO JIMENEZ'],
  'session_move_id': [53039, 'PV1A1/2024/04/0076 (POS/00166)']},
 {'id': 11242,
  'account_move': [52469, 'F1-CC/2024/04671 (PdV CSL/4628)'],
  'partner_id': [16087, 'VIDAL COSTICH CORTES'],
  'session_move_id': [53039, 'PV1A1/2024/04/0076 (POS/00166)']},
 {'id': 11244,
  'account_move': [52474, 'F1-CC/2024/04672 (PdV CSL/4629)'],
  'partner_id': [16253, 'Mostrador'],
  'session_move_id': [53039, 'PV1A1/2024/04/0076 (POS/00166)']},
 {'id': 11245,
  'account_move': [52478, 'F2-CC/2024/06237 (PdV SJC/6369)'],
  'partner_id': [16253, 'Mostrador'],
  'session_move_id': [53105, 'PV1A2/2024/04/0134 (POS/00167)']},
 {'id': 11246,
  'account_move': [52482, 'F2-CC/2024/06238 (PdV SJC/6370)'],
  'partner_id': [16253, 'Mostrador'],
  'session_move_id': [53105, 'PV1A2/2024/04/0134 (POS/00167)']},
 {'id': 11247,
  'account_move': [52487, 'F2-CC/2024/06239 (PdV SJC/6371)'

In [10]:
pay_pos_df

,pay_pos_id,pos_order_id,pos_order_name,date,amount
0,15934,11241,PdV CSL/4627,2024-04-01 15:27:40,2259.19
1,15935,11242,PdV CSL/4628,2024-04-01 15:33:46,2455.72
2,15937,11244,PdV CSL/4629,2024-04-01 15:36:24,437.87
3,15939,11245,PdV SJC/6369,2024-04-01 15:42:21,1815.32
4,15941,11246,PdV SJC/6370,2024-04-01 15:45:48,1709.16
...,...,...,...,...,...
2146,19303,13558,PdV SJC/7607,2024-04-18 15:13:00,269.00
2147,19304,13559,PdV CSL/5666,2024-04-20 04:38:59,10.00
2148,19306,13560,PdV CSL/5667,2024-04-20 05:25:50,50.00
2149,19308,13561,PdV SJC/7608,2024-04-20 05:34:21,5000.00


## Pruebas